<a href="https://colab.research.google.com/github/leupimik/fishapp_modeltraining/blob/main/ML_Modell_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import der einzelnen Bibliotheken

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf



In [20]:
import pathlib
dataset_url = "https://drive.google.com/file/d/1f3OFuX89ylxBSiH9UQ6ymBk0mm7gPuiI/view?usp=sharing"
data_dir = tf.keras.utils.get_file('Species', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

TypeError: ignored

Import der Bilder

In [17]:
_URL = 'https://drive.google.com/file/d/1f3OFuX89ylxBSiH9UQ6ymBk0mm7gPuiI/view?usp=sharing'

In [19]:
path_to_zip = tf.keras.utils.get_file('Species', origin=_URL, untar=True)

TypeError: ignored

In [12]:
PATH = os.path.join(os.path.dirname(_URL), 'Species')

In [13]:
train_dir = os.path.join(PATH, 'Training_Set')
validation_dir = os.path.join(PATH, 'Test_Set')

In [14]:
BATCH_SIZE = 32
IMG_SIZE = (255, 255)

In [15]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

NotFoundError: ignored

In [23]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Found 1000 files belonging to 2 classes.


Show the first nine images and labels from the training set:

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")